# Data Augmentation

In [1]:
import os
import cv2
import glob
import keras
from keras.utils.image_utils import img_to_array, array_to_img, load_img
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#STARE
def data_augmentation():
    img_dir = '../data/filtered-data/ODIR/Pure AMD' #orginal directory
    aug_dir = '../data/filtered-data/ODIR/Pure AMD/Augmented' #augmented directory
    data_path = os.path.join(img_dir,'*g')

    files = glob.glob(data_path) #error @ rgb_img when removing glob.glob dunno y
    data = []
    for f1 in files:
        img = cv2.imread(f1) #read img
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #convert BGR img to RGB (depends)
        data.append(rgb_img) 

        x = img_to_array(rgb_img)  
        x = x.reshape((1,) + x.shape)  

        path, dirs, files = next(os.walk(img_dir))
        file_count = len(files)

        datagen = ImageDataGenerator(horizontal_flip = True)

        i = 0
        for batch in datagen.flow(x, batch_size = 1, save_to_dir = aug_dir, save_prefix = 'ODIR_Augment', save_format = 'jpg'):
            i += 1
            if i == 1:
                break  

# Red Channel Extraction & CLAHE

In [3]:
import numpy as np
import os
from os import listdir

In [4]:
amd_w_others = []
normal = []
pure_amd = []

In [5]:
def rce_clahe(img_dir, red_dir, data_family, data_class):
    if not os.path.exists(red_dir):
        os.mkdir(red_dir)

    datagen = ImageDataGenerator(horizontal_flip = True)

    i = 0
    for images in os.listdir(img_dir): 
        # check if the image ends with png or jpg or jpeg
        if (images.endswith(".png") or images.endswith(".jpg")\
            or images.endswith(".jpeg")):
            img = cv2.imread(img_dir + "/" + images, cv2.IMREAD_UNCHANGED)
            resized_img = cv2.resize(img, (512, 512))

            # exclude red channel
            b,g,r = cv2.split(resized_img)

            # apply CLAHE
            clahe = cv2.createCLAHE(clipLimit=5)
            b_clahe_enhanced = clahe.apply(b) + 30 # increase intensity of contrast
            g_clahe_enhanced = clahe.apply(g) + 30
            r_clahe_enhanced = clahe.apply(r)

            # merge enhanced channels
            clahe_enhanced_img = cv2.merge((b_clahe_enhanced, g, r_clahe_enhanced))

            grayscale = cv2.cvtColor(clahe_enhanced_img, cv2.COLOR_BGR2GRAY)

            if data_class == 'pure_amd':
                pure_amd.append(grayscale)
            elif data_class == 'normal':
                normal.append(grayscale)
            elif data_class == 'amd_w_others':
                amd_w_others.append(grayscale)

            path = os.path.join(red_dir, "{}_CLAHE_".format(data_family) + images.split('.')[0] + ".png")
            cv2.imwrite(path , grayscale)
            i+=1

In [6]:
rce_clahe('../data/preprocessing/filtered-data/ODIR/Pure AMD', '../data/preprocessing/CLAHE/ODIR/Pure AMD', 'ODIR', 'pure_amd')
rce_clahe('../data/preprocessing/filtered-data/ODIR/AMD w others', '../data/preprocessing/CLAHE/ODIR/AMD w others', 'ODIR', 'amd_w_others')
rce_clahe('../data/preprocessing/filtered-data/ODIR/Normal', '../data/preprocessing/CLAHE/ODIR/Normal', 'ODIR', 'normal')

In [7]:
rce_clahe('../data/preprocessing/filtered-data/RFMiD/Pure AMD', '../data/preprocessing/CLAHE/RFMiD/Pure AMD', 'RFMiD', 'pure_amd')
rce_clahe('../data/preprocessing/filtered-data/RFMiD/AMD w others', '../data/preprocessing/CLAHE/RFMiD/AMD w others', 'RFMiD', 'amd_w_others')
rce_clahe('../data/preprocessing/filtered-data/RFMiD/Normal', '../data/preprocessing/CLAHE/RFMiD/Normal', 'RFMiD', 'normal')

In [8]:
rce_clahe('../data/preprocessing/filtered-data/STARE/AMD w others', '../data/preprocessing/CLAHE/STARE/AMD w others', 'STARE', 'amd_w_others')
rce_clahe('../data/preprocessing/filtered-data/STARE/Normal', '../data/preprocessing/CLAHE/STARE/Normal', 'STARE', 'normal')

# Segmentation

In [9]:
from sklearn.model_selection import train_test_split
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [10]:
def unet(pretrained_weights=None, input_size=(512,512,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(learning_rate=1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [12]:
pure_amd

[array([[6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        ...,
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6]], dtype=uint8),
 array([[6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        ...,
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6]], dtype=uint8),
 array([[6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        ...,
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6]], dtype=uint8),
 array([[6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        ...,
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6]], dtype=uint8),
 array([[6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6],
        [6, 6, 6, ..., 6, 6, 6

In [25]:
x = np.array(pure_amd).reshape(-1, 512, 512, 1)

In [29]:
y = []

for i in range(len(x)):
    y.append(1)

In [30]:
model = unet()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_72 (Conv2D)             (None, 512, 512, 64  640         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_73 (Conv2D)             (None, 512, 512, 64  36928       ['conv2d_72[0][0]']              
                                )                                                           

In [31]:
model.fit(x, y, steps_per_epoch=2000, epochs=5)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})